<a href="https://colab.research.google.com/github/jisoo71/PedTalk/blob/main/gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.5/324.5 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 111.1 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.10.1
    Uninstalling gradio_client-1.10.1:
      Successfully uninstalled gradio_client-1.10.1
  Attempting uninstall: gradio
    Found existing installation: gradio 5.31.0
    Uninstalling gradio-5.31.0:
      Successfully uninstalled gradio-5.31.0


In [ ]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
import gradio as gr

# -----------------------
# 1) 더미 학습 데이터 준비 (영어 컬럼명)
# -----------------------
np.random.seed(42)

n = 100
df = pd.DataFrame({
    "age_month": np.random.randint(1, 150, n),
    "current_temp": np.random.uniform(36.5, 40.5, n).round(1),
    "cough": np.random.randint(0, 2, n),
    "runny_nose": np.random.randint(0, 2, n),
    "diarrhea": np.random.randint(0, 2, n),
    "convulsion": np.random.randint(0, 2, n),
    "poor_appetite": np.random.randint(0, 2, n),
    "sore_throat": np.random.randint(0, 2, n),
    "rash": np.random.randint(0, 2, n),
    "altered_consciousness": np.random.randint(0, 2, n),
    "under_3_month": np.random.randint(0, 2, n),
})

priority = np.random.choice(["low(※)", "medium(※※)", "high(※※※)", "very high(※※※※)"], n)
diagnosis = np.random.choice(["flu", "common cold", "gastroenteritis", "hand foot mouth", "febrile seizure"], n)

df["SVM_priority"] = priority
df["predicted_diagnosis"] = diagnosis

# -----------------------
# 2) 레이블 변환
# -----------------------
priority_map = {"low(※)": 0, "medium(※※)": 1, "high(※※※)": 2, "very high(※※※※)": 3}
df["priority_label"] = df["SVM_priority"].map(priority_map)

diagnosis_encoder = LabelEncoder()
df["diagnosis_label"] = diagnosis_encoder.fit_transform(df["predicted_diagnosis"])

features = ["age_month", "current_temp", "cough", "runny_nose", "diarrhea", "convulsion",
            "poor_appetite", "sore_throat", "rash", "altered_consciousness", "under_3_month"]

X = df[features].values
y_priority = df["priority_label"].values
y_diagnosis = df["diagnosis_label"].values

# -----------------------
# 3) 모델 학습
# -----------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

svm_model = SVC(kernel="rbf", probability=True)
svm_model.fit(X_scaled, y_priority)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_scaled, y_diagnosis)

# -----------------------
# 4) Gradio 예측 함수
# -----------------------
priority_reverse_map = {0: "low(※)", 1: "medium(※※)", 2: "high(※※※)", 3: "very high(※※※※)"}

def predict_both(csv_file):
    df_input = pd.read_csv(csv_file.name)
    X_input = df_input[features].values
    X_scaled = scaler.transform(X_input)

    preds_priority = svm_model.predict(X_scaled)
    preds_diagnosis = rf_model.predict(X_scaled)

    df_input["SVM_priority"] = [priority_reverse_map[p] for p in preds_priority]
    df_input["predicted_diagnosis"] = diagnosis_encoder.inverse_transform(preds_diagnosis)

    return df_input

# -----------------------
# 5) Gradio 실행
# -----------------------
interface = gr.Interface(
    fn=predict_both,
    inputs=gr.File(file_types=['.csv']),
    outputs=gr.Dataframe(),
    title="✅ Medical Priority and Diagnosis Prediction",
    description="Upload CSV and get SVM priority + RandomForest diagnosis predictions."
)

interface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://68fa98ad0afb98b069.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
from sklearn import svm
import numpy as np

# 예시 모델 (실제 모델로 대체)
model = svm.SVC()
X_train = np.array([[1, 0], [0, 1], [1, 1], [0, 0]])
y_train = np.array([1, 1, 0, 0])
model.fit(X_train, y_train)

def predict_triage(fever, cough):
    X = np.array([[fever, cough]])
    prediction = model.predict(X)[0]
    return "고위험군" if prediction == 1 else "저위험군"

demo = gr.Interface(
    fn=predict_triage,
    inputs=[gr.Slider(0, 1, label="발열"), gr.Slider(0, 1, label="기침")],
    outputs="text",
    title="AI 문진 기반 우선순위 예측 시뮬레이터"
)

demo.launch()


In [ ]:
# 1. 설치
!pip install -q gradio

# 2. 필요한 패키지 불러오기
import gradio as gr
import numpy as np
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# 3. 원본 데이터 구성 (너의 코드 그대로 반영)
X = np.array([
    [12, 38.5, 0, 0, 0, 0],
    [6, 39.2, 0, 0, 0, 0],
    [2, 38.8, 0, 0, 0, 1],
    [24, 39.8, 1, 0, 0, 0],
    [18, 39.5, 0, 1, 0, 0],
    [5, 40.1, 0, 0, 1, 1],
    [60, 39.0, 0, 0, 0, 0],
    [10, 39.0, 0, 0, 0, 0],
    [7, 38.0, 1, 0, 0, 0],
    [1, 39.0, 0, 0, 0, 1],
])
y = np.array([0, 1, 1, 2, 3, 3, 0, 1, 2, 3])

# 4. 학습 (너 코드 기반)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
svm_model = svm.SVC(kernel='rbf', C=1, probability=True)
svm_model.fit(X_train_scaled, y_train)

# 5. 예측 함수 정의
def predict_priority_gradio(age_months, temp, resp, conv, conc):
    is_infant = 1 if age_months < 3 else 0
    features = np.array([[age_months, temp, resp, conv, conc, is_infant]])
    scaled_features = scaler.transform(features)
    pred = svm_model.predict(scaled_features)[0]
    priority_map = {0: "※ (낮음)", 1: "※※ (보통)", 2: "※※※ (높음)", 3: "※※※※ (매우 높음)"}
    return priority_map[pred]

# 6. Gradio GUI 구성
interface = gr.Interface(
    fn=predict_priority_gradio,
    inputs=[
        gr.Slider(0, 144, value=12, step=1, label="연령 (개월)"),
        gr.Slider(35.0, 41.0, value=38.5, step=0.1, label="현재 체온 (도)"),
        gr.Radio([0, 1], label="호흡곤란 (0=없음, 1=있음)", value=0),
        gr.Radio([0, 1], label="경련 (0=없음, 1=있음)", value=0),
        gr.Radio([0, 1], label="의식변화 (0=없음, 1=있음)", value=0),
    ],
    outputs=gr.Textbox(label="예측된 진료 우선순위"),
    title="SVM 기반 진료 우선순위 시뮬레이터",
    description="연령, 체온, 증상 등을 입력하면 우선순위를 예측합니다.",
    theme="soft"
)

# 실행
interface.launch()


# 시각화

In [ ]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
import gradio as gr
import plotly.express as px

# -----------------------
# 1) 더미 학습 데이터 준비
# -----------------------
np.random.seed(42)
n = 100

df = pd.DataFrame({
    "age_month": np.random.randint(1, 150, n),
    "current_temp": np.random.uniform(36.5, 40.5, n).round(1),
    "cough": np.random.randint(0, 2, n),
    "runny_nose": np.random.randint(0, 2, n),
    "diarrhea": np.random.randint(0, 2, n),
    "convulsion": np.random.randint(0, 2, n),
    "poor_appetite": np.random.randint(0, 2, n),
    "sore_throat": np.random.randint(0, 2, n),
    "rash": np.random.randint(0, 2, n),
    "altered_consciousness": np.random.randint(0, 2, n),
    "under_3_month": np.random.randint(0, 2, n),
})

priority = np.random.choice(["low(※)", "medium(※※)", "high(※※※)", "very high(※※※※)"], n)
diagnosis = np.random.choice(["flu", "common cold", "gastroenteritis", "hand foot mouth", "febrile seizure"], n)

df["SVM_priority"] = priority
df["predicted_diagnosis"] = diagnosis

# -----------------------
# 2) Label Encoding
# -----------------------
priority_map = {"low(※)": 0, "medium(※※)": 1, "high(※※※)": 2, "very high(※※※※)": 3}
priority_reverse_map = {0: "low(※)", 1: "medium(※※)", 2: "high(※※※)", 3: "very high(※※※※)"}

df["priority_label"] = df["SVM_priority"].map(priority_map)

diagnosis_encoder = LabelEncoder()
df["diagnosis_label"] = diagnosis_encoder.fit_transform(df["predicted_diagnosis"])

features = ["age_month", "current_temp", "cough", "runny_nose", "diarrhea",
            "convulsion", "poor_appetite", "sore_throat", "rash",
            "altered_consciousness", "under_3_month"]

X = df[features].values
y_priority = df["priority_label"].values
y_diagnosis = df["diagnosis_label"].values

# -----------------------
# 3) 모델 학습
# -----------------------
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

svm_model = SVC(kernel="rbf", probability=True)
svm_model.fit(X_scaled, y_priority)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_scaled, y_diagnosis)

# -----------------------
# 4) Gradio 함수
# -----------------------
def predict_both(csv_file):
    df_input = pd.read_csv(csv_file.name)
    X_input = df_input[features].values
    X_scaled_input = scaler.transform(X_input)

    preds_priority = svm_model.predict(X_scaled_input)
    preds_diagnosis = rf_model.predict(X_scaled_input)

    df_input["SVM_priority"] = [priority_reverse_map[p] for p in preds_priority]
    df_input["predicted_diagnosis"] = diagnosis_encoder.inverse_transform(preds_diagnosis)

    display_df = df_input[features + ["SVM_priority", "predicted_diagnosis"]]

    # 그래프: 우선순위
    priority_counts = display_df["SVM_priority"].value_counts().reset_index()
    priority_counts.columns = ["Priority", "Count"]
    fig_priority = px.bar(priority_counts, x="Priority", y="Count", title="Priority Distribution")

    # 그래프: 진단
    fig_diag = px.pie(display_df, names="predicted_diagnosis", title="Diagnosis Distribution")

    # 그래프: 증상
    symptom_cols = ["cough", "runny_nose", "diarrhea", "convulsion",
                    "poor_appetite", "sore_throat", "rash", "altered_consciousness"]
    symptom_counts = display_df[symptom_cols].sum().sort_values(ascending=False).reset_index()
    symptom_counts.columns = ["Symptom", "Count"]

    fig_symptom = px.bar(symptom_counts, x="Symptom", y="Count", title="Symptom Occurrences")

    return display_df, fig_priority, fig_diag, fig_symptom

# -----------------------
# 5) Gradio 실행
# -----------------------
interface = gr.Interface(
    fn=predict_both,
    inputs=gr.File(file_types=['.csv']),
    outputs=[
        gr.Dataframe(label="Prediction Results"),
        gr.Plot(label="Priority Distribution"),
        gr.Plot(label="Diagnosis Distribution"),
        gr.Plot(label="Symptom Occurrences"),
    ],
    description="Upload CSV file to see prediction results with summary charts."
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://34ab03762033a90210.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
